In [10]:
#import modules
import pandas as pd
import numpy as np 
import seaborn as sb
import pandas_profiling
import matplotlib.pyplot as plt
import sqlalchemy as db
from sqlalchemy import MetaData, Table

In [8]:
#connecting to database
engine = db.create_engine("sqlite:///im.db")

In [39]:
inspector = db.inspect(engine)
table_names = inspector.get_table_names()

In [40]:
metadata = MetaData()
connection = engine.connect()
all_data = []

for i in table_names:
    t = Table(i, metadata, autoload = True, autoload_with=engine )
    query = db.select([t])
    ResultProxy = connection.execute(query)
    ResultSet = ResultProxy.fetchall()
    all_data.append(ResultSet)

In [42]:
len(all_data)

8

In [21]:
metadata = MetaData()
connection = engine.connect()
movie_ratings = Table("movie_basics", metadata, autoload=True, autoload_with=engine)

In [23]:
query = db.select([movie_ratings])
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()


In [25]:
df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()

In [37]:
ResultSet

[('tt0063540', 'Sunghursh', 'Sunghursh', 2013, 175.0, 'Action,Crime,Drama'),
 ('tt0066787', 'One Day Before the Rainy Season', 'Ashad Ka Ek Din', 2019, 114.0, 'Biography,Drama'),
 ('tt0069049', 'The Other Side of the Wind', 'The Other Side of the Wind', 2018, 122.0, 'Drama'),
 ('tt0069204', 'Sabse Bada Sukh', 'Sabse Bada Sukh', 2018, None, 'Comedy,Drama'),
 ('tt0100275', 'The Wandering Soap Opera', 'La Telenovela Errante', 2017, 80.0, 'Comedy,Drama,Fantasy'),
 ('tt0111414', 'A Thin Life', 'A Thin Life', 2018, 75.0, 'Comedy'),
 ('tt0112502', 'Bigfoot', 'Bigfoot', 2017, None, 'Horror,Thriller'),
 ('tt0137204', 'Joe Finds Grace', 'Joe Finds Grace', 2017, 83.0, 'Adventure,Animation,Comedy'),
 ('tt0139613', 'O Silêncio', 'O Silêncio', 2012, None, 'Documentary,History'),
 ('tt0144449', 'Nema aviona za Zagreb', 'Nema aviona za Zagreb', 2012, 82.0, 'Biography'),
 ('tt0146592', 'Pál Adrienn', 'Pál Adrienn', 2010, 136.0, 'Drama'),
 ('tt0154039', 'So Much for Justice!', 'Oda az igazság', 2010, 10

In [27]:
df.isnull().sum()

movie_id               0
primary_title          0
original_title        21
start_year             0
runtime_minutes    31739
genres              5408
dtype: int64